This code make necessary adjustments to the code in 1. Run 1a immediately after running 1. 

In [21]:
import os
import getpass
import sys
current_user = getpass.getuser()
sys.path.append(f"C:/Users/{current_user}/OneDrive - The Pennsylvania State University/2207 - Zoning - exercise")
from config import *
import geopandas as gpd
import pandas as pd
mapc_sf = gpd.read_file(mapc_shp)
for index, row in mapc_sf.iterrows():
    polygon = row['geometry']
    mapc_sf.loc[index, 'geometry'] = polygon.convex_hull
mapc_sf = mapc_sf.to_crs(epsg=4326)

gdf = gpd.read_file(tract2019_shp)
centroid_list = gdf[['STATEFP','COUNTYFP','TRACTCE','INTPTLAT','INTPTLON']]
centroid_gdf = gpd.GeoDataFrame(centroid_list, geometry=gpd.points_from_xy(centroid_list["INTPTLON"], centroid_list["INTPTLAT"]))
centroid_gdf = centroid_gdf.set_crs(epsg=4326)

geodf = gpd.sjoin(centroid_gdf, mapc_sf, predicate='intersects', how = "inner")

geodf_list = geodf[['STATEFP','COUNTYFP','TRACTCE','INTPTLAT','INTPTLON']].drop_duplicates()


In [25]:

old_geodf_list = pd.read_csv(tract_ma_2019_csv)

int_columns = ['STATEFP', 'COUNTYFP', 'TRACTCE']
geodf_list[int_columns] = geodf_list[int_columns].astype(int)
geodf_list['INTPTLAT'] = geodf_list['INTPTLAT'].astype(float)
geodf_list['INTPTLON'] = geodf_list['INTPTLON'].astype(float)
full_geodf_list = old_geodf_list.merge(geodf_list, on = ['STATEFP','COUNTYFP','TRACTCE','INTPTLAT','INTPTLON'], how = 'outer').sort_values(by='tract_index')

N = len(full_geodf_list)
full_geodf_list['tract_index'] = range(N)

print(full_geodf_list)

     STATEFP  COUNTYFP  TRACTCE   INTPTLAT   INTPTLON  tract_index
0         25        25   170501  42.415033 -70.990222            0
1         25        25   170702  42.406567 -70.997479            1
2         25        27   715100  42.439933 -71.601885            2
3         25        21   418004  42.238376 -71.010499            3
4         25        21   417902  42.246302 -70.990692            4
..       ...       ...      ...        ...        ...          ...
700       25        21   422302  42.189154 -70.939244          700
701       25        17   363202  42.479293 -71.466853          701
702       25        21   410400  42.087421 -71.251974          702
703       25        21   408102  42.146466 -71.415591          703
704       25        21   423100  42.229395 -70.827123          704

[705 rows x 6 columns]


In [26]:
int_columns = ['STATEFP', 'COUNTYFP', 'TRACTCE']
gdf[int_columns] = gdf[int_columns].astype(int)
gdf['INTPTLAT'] = gdf['INTPTLAT'].astype(float)
gdf['INTPTLON'] = gdf['INTPTLON'].astype(float)

merged_gdf = gdf.merge(full_geodf_list, on=['STATEFP','COUNTYFP','TRACTCE','INTPTLAT','INTPTLON'], how='inner')
merged_gdf.to_file(tract2019_bo_shp)

full_geodf_list.to_csv(tract_ma_2019_csv,index = False)

C:\Users\local_dmh5950\Temp\ipykernel_21840\2126735139.py:7: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  merged_gdf.to_file(tract2019_bo_shp)
